In [1]:
import os
import matplotlib.pyplot as plt
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch, torchvision
import detectron2
import json
import ast 
import tensorboard,os
from detectron2.utils.logger import setup_logger

setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [2]:
files_to_remove = os.listdir('./output/')
files_to_remove.remove('metrics.json')
for file_to_remove in files_to_remove:
    os.remove(f'./output/{file_to_remove}')

In [3]:
data = pd.read_csv("./Data.csv").sample(frac=1)

In [4]:
info = data.iloc[59]
img = cv2.imread(f'./Img/{info["Path"]}')
height, width = cv2.imread("./Img/" + info["Path"]).shape[:2]
xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
xmin = round(xmin * width)
xmax = round(xmax * width)
ymin = round(ymin * height)
ymax = round(ymax * height)
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
x,y,w,h = round(x),round(y),round(w),round(h)
cv2.imwrite('./output.png',img)
roi=img[y:y+h,x:x+w]
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),10)

array([[[187, 211, 217],
        [185, 209, 215],
        [185, 209, 215],
        ...,
        [191, 212, 220],
        [194, 215, 223],
        [195, 216, 224]],

       [[187, 211, 217],
        [186, 210, 216],
        [186, 210, 216],
        ...,
        [196, 215, 223],
        [195, 214, 222],
        [195, 214, 222]],

       [[190, 212, 218],
        [189, 211, 217],
        [187, 211, 217],
        ...,
        [197, 216, 224],
        [196, 213, 222],
        [197, 214, 223]],

       ...,

       [[ 37,  29,  12],
        [ 35,  27,  10],
        [ 36,  28,  11],
        ...,
        [ 65,  41,  17],
        [ 61,  41,  16],
        [ 61,  41,  16]],

       [[ 39,  29,  12],
        [ 37,  27,  10],
        [ 39,  29,  12],
        ...,
        [ 67,  43,  21],
        [ 61,  40,  18],
        [ 61,  40,  18]],

       [[ 40,  30,  13],
        [ 38,  28,  11],
        [ 40,  30,  13],
        ...,
        [ 68,  44,  22],
        [ 65,  41,  21],
        [ 63,  42,  21]]

In [5]:
# Loading Data
def load_data(data=data, test=False):
    if test is True:
        if "data.npy" in os.listdir("./"):
            data = np.load("./data.npy", allow_pickle=True)
            data = data[:325]
            print(len(data))
            return data
    if "data.npy" in os.listdir("./"):
        data = np.load("./data.npy", allow_pickle=True)
        print(len(data))
        return data
    new_data = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        height, width = cv2.imread("./Img/" + info["Path"]).shape[:2]
        xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
        xmin = round(xmin * width)
        xmax = round(xmax * width)
        ymin = round(ymin * height)
        ymax = round(ymax * height)
        record["file_name"] = "./Img/" + info["Path"]
        record["height"] = height
        record["width"] = width
        objs = [
            {
                "bbox": [xmin,ymin,xmax,ymax],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 0,
            }
        ]
        record["image_id"] = idx
        record["annotations"] = objs
        new_data.append(record)
    np.random.shuffle(new_data)
    np.save("data.npy", new_data)
    return new_data

In [6]:
# Config
labels = ["Card"]

In [7]:
# Adding the data
DatasetCatalog.register("data", lambda: load_data())
MetadataCatalog.get("data").set(thing_classes=labels)
metadata = MetadataCatalog.get("data")
DatasetCatalog.register("test", lambda: load_data(test=True))
MetadataCatalog.get("test").set(thing_classes=labels)
metadata_test = MetadataCatalog.get("test")

In [8]:
models = [
    "fast_rcnn_R_50_FPN_1x.yaml",
    "faster_rcnn_R_50_C4_1x.yaml",
    "faster_rcnn_R_50_C4_3x.yaml",
    "faster_rcnn_R_50_DC5_1x.yaml",
    "faster_rcnn_R_50_DC5_3x.yaml",
    "retinanet_R_50_FPN_1x.py",
    "retinanet_R_50_FPN_1x.yaml",
    "retinanet_R_50_FPN_3x.yaml",
    "rpn_R_50_C4_1x.yaml",
    "rpn_R_50_FPN_1x.yaml"
    "faster_rcnn_R_50_FPN_1x.yaml",
    "faster_rcnn_R_50_FPN_3x.yaml",
    "faster_rcnn_R_101_DC5_3x.yaml",
    "faster_rcnn_R_101_FPN_3x.yaml",
    "faster_rcnn_X_101_32x8d_FPN_3x.yaml",
]
max_iters = [
    50,100,125,250,500,1000,2000,2500,5000
]
base_lrs = [
    0.1,0.01,0.001,0.0001,0.00001,0.000001
]
ims_per_batchs = [
    1,2,3,4,5,6,7,8,9,10
]
batch_size_per_images = [
    8,16,32,64,128,256,512
]

In [9]:
BASE_LR = 0.00025
MAX_ITER = 500
EVAL_PERIOD = 500
IMS_PER_BATCH = 2
BATCH_SIZE_PER_IMAGE = 128
SCORE_THRESH_TEST = 0.625
model = f"COCO-Detection/" + "faster_rcnn_X_101_32x8d_FPN_3x.yaml"

In [10]:
NAME = "test"

In [11]:
files_to_remove = os.listdir("./output/")
for file_to_remove in files_to_remove:
    os.remove(f"./output/{file_to_remove}")
from detectron2.utils.logger import setup_logger
setup_logger()
torch.cuda.empty_cache()
wandb.init(project="Find-Card", name=NAME,config={
    'BASE_LR':BASE_LR,
    'MAX_ITER':MAX_ITER,
    'EVAL_PERIOD':EVAL_PERIOD,
    'IMS_PER_BATCH':IMS_PER_BATCH,
    'BATCH_SIZE_PER_IMAGE':BATCH_SIZE_PER_IMAGE,
    'SCORE_THRESH_TEST':SCORE_THRESH_TEST,
    'MODEL':model,
    'NAME':NAME
})
torch.cuda.empty_cache()
cfg = get_cfg()
torch.cuda.empty_cache()
cfg.merge_from_file(model_zoo.get_config_file(model))
torch.cuda.empty_cache()
cfg.DATASETS.TRAIN = ("data",)
torch.cuda.empty_cache()
cfg.DATASETS.TEST = ()
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
torch.cuda.empty_cache()
cfg.SOLVER.MAX_ITER = MAX_ITER
torch.cuda.empty_cache()
cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.SOLVER.BASE_LR = BASE_LR
torch.cuda.empty_cache()
cfg.SOLVER.STEPS = []
torch.cuda.empty_cache()
cfg.SOLVER.IMS_PER_BATCH = IMS_PER_BATCH
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = BATCH_SIZE_PER_IMAGE
torch.cuda.empty_cache()
trainer = DefaultTrainer(cfg)
torch.cuda.empty_cache()
trainer.resume_or_load(resume=False)
torch.cuda.empty_cache()
trainer.train()
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = SCORE_THRESH_TEST
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
torch.cuda.empty_cache()
predictor = DefaultPredictor(cfg)
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
cfg.SOLVER.SCORE_THRESH_TEST = SCORE_THRESH_TEST
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("test", output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test")
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
wandb.log(metrics)
torch.cuda.empty_cache()
logs = open("./output/metrics.json", "r").read().split("\n")
for log in tqdm(range(len(logs))):
    try:
        res = ast.literal_eval(logs[log])
        wandb.log(res)
    except:
        pass
for img in os.listdir("./test_imgs/"):
    torch.cuda.empty_cache()
    v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
    torch.cuda.empty_cache()
    v = v.draw_instance_predictions(
        predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
    )
    torch.cuda.empty_cache()
    v = v.get_image()[:, :, ::-1]
    torch.cuda.empty_cache()
    plt.figure(figsize=(24, 12))
    torch.cuda.empty_cache()
    plt.imshow(v)
    torch.cuda.empty_cache()
    plt.savefig(f"./preds/{img}")
    torch.cuda.empty_cache()
    plt.close()
    torch.cuda.empty_cache()
    wandb.log({f"Img/{img}": wandb.Image(cv2.imread(f"./preds/{img}"))})
info = data.iloc[589]
img = cv2.imread("./download/Img/" + info["Path"])
height, width = cv2.imread("./download/Img/" + info["Path"]).shape[:2]
xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
xmin = round(xmin * width)
xmax = round(xmax * width)
ymin = round(ymin * height)
ymax = round(ymax * height)
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
preds = predictor(img)
target = torch.tensor([xmin, ymin, xmax, ymax])
lowest_rmse = 0
r_mean_squared_error = MeanSquaredError(squared=False)
preds_new = preds["instances"].__dict__["_fields"]["pred_boxes"].__dict__["tensor"]
for pred_i in range(len(preds)):
    pred = preds_new[pred_i]
    if r_mean_squared_error(pred.to("cpu"), target) > lowest_rmse:
        lowest_rmse = r_mean_squared_error(pred.to("cpu"), target)
lowest_mse = 0
mean_squared_error = MeanSquaredError(squared=True)
preds_new = preds["instances"].__dict__["_fields"]["pred_boxes"].__dict__["tensor"]
for pred_i in range(len(preds)):
    pred = preds_new[pred_i]
    if mean_squared_error(pred.to("cpu"), target) > lowest_mse:
        lowest_mse = mean_squared_error(pred.to("cpu"), target)
wandb.log({"MSE": lowest_mse})
wandb.log({"RMSE": lowest_rmse})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-28 20:45:01.168085: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


[10/28 20:45:11 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

2021-10-28 20:45:11.909777: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model p

[10/28 20:45:15 d2.engine.train_loop]: Starting training from iteration 0


/media/indika/Sync/anaconda3/envs/detectron2/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[10/28 20:45:35 d2.utils.events]:  eta: 0:07:35  iter: 19  total_loss: 1.027  loss_cls: 0.6055  loss_box_reg: 0.3171  loss_rpn_cls: 0.02086  loss_rpn_loc: 0.006698  time: 0.9913  data_time: 0.0334  lr: 9.7405e-06  max_mem: 5819M
[10/28 20:45:57 d2.utils.events]:  eta: 0:07:32  iter: 39  total_loss: 0.9201  loss_cls: 0.5126  loss_box_reg: 0.3697  loss_rpn_cls: 0.01318  loss_rpn_loc: 0.007745  time: 1.0443  data_time: 0.0024  lr: 1.9731e-05  max_mem: 5851M
[10/28 20:46:22 d2.utils.events]:  eta: 0:07:27  iter: 59  total_loss: 0.8185  loss_cls: 0.4076  loss_box_reg: 0.3843  loss_rpn_cls: 0.01381  loss_rpn_loc: 0.009916  time: 1.1067  data_time: 0.0025  lr: 2.972e-05  max_mem: 6043M
[10/28 20:46:43 d2.utils.events]:  eta: 0:07:07  iter: 79  total_loss: 0.8034  loss_cls: 0.3462  loss_box_reg: 0.4164  loss_rpn_cls: 0.01552  loss_rpn_loc: 0.005157  time: 1.0902  data_time: 0.0023  lr: 3.9711e-05  max_mem: 6043M
[10/28 20:47:03 d2.utils.events]:  eta: 0:06:38  iter: 99  total_loss: 0.7256  los

100%|█████████████████████████████████████████| 26/26 [00:00<00:00, 1749.93it/s]


AttributeError: 'NoneType' object has no attribute 'shape'

In [14]:
from torchmetrics import MeanAbsoluteError

In [ ]:
info = data.iloc[589]
img = cv2.imread("./download/Img/" + info["Path"])
height, width = cv2.imread("./download/Img/" + info["Path"]).shape[:2]
xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
xmin = round(xmin * width)
xmax = round(xmax * width)
ymin = round(ymin * height)
ymax = round(ymax * height)
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
preds = predictor(img)
target = torch.tensor([xmin, ymin, xmax, ymax])
lowest_rmse = 0
r_mean_squared_error = MeanSquaredError(squared=False)
preds_new = preds["instances"].__dict__["_fields"]["pred_boxes"].__dict__["tensor"]
for pred_i in range(len(preds)):
    pred = preds_new[pred_i]
    if r_mean_squared_error(pred.to("cpu"), target) > lowest_rmse:
        lowest_rmse = r_mean_squared_error(pred.to("cpu"), target)
lowest_mse = 0
mean_squared_error = MeanSquaredError(squared=True)
preds_new = preds["instances"].__dict__["_fields"]["pred_boxes"].__dict__["tensor"]
for pred_i in range(len(preds)):
    pred = preds_new[pred_i]
    if mean_squared_error(pred.to("cpu"), target) > lowest_mse:
        lowest_mse = mean_squared_error(pred.to("cpu"), target)
wandb.log({"MSE": lowest_mse})
wandb.log({"RMSE": lowest_rmse})
wandb.finish()

In [12]:
info['Path']

'625.png'

In [ ]:
torch.save(cfg,f'./models/cfg-{NAME}.pt')
torch.save(cfg,f'./models/cfg-{NAME}.pth')
torch.save(predictor,f'./models/predictor-{NAME}.pt')
torch.save(predictor,f'./models/predictor-{NAME}.pth')
torch.save(evaluator,f'./models/evaluator-{NAME}.pt')
torch.save(evaluator,f'./models/evaluator-{NAME}.pth')
torch.save(model,f'./models/model-{NAME}.pt')
torch.save(model,f'./models/model-{NAME}.pth')
torch.save(labels,f'./models/labels-{NAME}.pt')
torch.save(labels,f'./models/labels-{NAME}.pth')
torch.save(metrics,f'./models/metrics-{NAME}.pt')
torch.save(metrics,f'./models/metrics-{NAME}.pth')